In [1]:
import findspark

findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import *

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
(
spark
    .read
    .csv('title_ratings.tsv', header=True, sep='\t')
    .write
    .format('parquet')
    .mode('overwrite')
    .save('dfratings')
)

In [5]:
(
spark
    .read
    .csv('title_basics.tsv', header=True, sep='\t')
    .write
    .format('parquet')
    .mode('overwrite')
    .save('dfbasics')
)

In [6]:
df_basics = spark.read.format('parquet').load('dfbasics')

In [10]:
df_basics.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)



In [7]:
df_basics.show(5)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000001|    short|          Carmencita|          Carmencita|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      Pauvre Pierrot|      0|     1892|     \N|             4|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|    Blacksmith Scene|      0|     1893|     \N|             1|        Comedy

In [8]:
df_ratings = spark.read.format('parquet').load('dfratings')

In [11]:
df_ratings.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- averageRating: string (nullable = true)
 |-- numVotes: string (nullable = true)



In [9]:
df_ratings.show(5)

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    1809|
|tt0000002|          6.0|     233|
|tt0000003|          6.5|    1560|
|tt0000004|          6.1|     152|
|tt0000005|          6.2|    2383|
+---------+-------------+--------+
only showing top 5 rows



<h3>Respondendo as questões do trabalho prático</h3>

In [21]:
df_basics.withColumn('startYear', col('startYear').cast('int'))

DataFrame[tconst: string, titleType: string, primaryTitle: string, originalTitle: string, isAdult: string, startYear: int, endYear: string, runtimeMinutes: string, genres: string]

In [22]:
df_basics.createOrReplaceTempView('df_basics')

In [28]:
spark.sql("SELECT DISTINCT titleType FROM df_basics").show()

+------------+
|   titleType|
+------------+
|    tvSeries|
|tvMiniSeries|
|     tvMovie|
|   tvEpisode|
|       movie|
|   tvSpecial|
|       video|
|   videoGame|
|     tvShort|
|       short|
|     tvPilot|
| radioSeries|
|radioEpisode|
+------------+



In [35]:
spark.sql("SELECT COUNT(DISTINCT primaryTitle) FROM df_basics WHERE startYear = 2015 AND titleType in ('tvMovie', 'movie')").show(5)

+----------------------------+
|count(DISTINCT primaryTitle)|
+----------------------------+
|                       19668|
+----------------------------+



In [45]:
spark.sql('''SELECT genre, COUNT(genre) AS g FROM (SELECT SPLIT(genres,',')[0] AS genre
             FROM   df_basics) group by genre ORDER BY g DESC''').show()

+-----------+-------+
|      genre|      g|
+-----------+-------+
|      Drama|1654014|
|     Comedy|1415652|
|Documentary| 660862|
|         \N| 643012|
|  Talk-Show| 467793|
|       News| 432325|
|     Action| 334580|
| Reality-TV| 269848|
|  Animation| 260424|
|      Crime| 254639|
|      Adult| 242249|
|     Family| 233217|
|      Music| 224636|
|  Adventure| 200780|
|      Short| 199968|
|  Game-Show| 165183|
|    Romance|  99870|
|      Sport|  99780|
|  Biography|  81676|
|     Horror|  79556|
+-----------+-------+
only showing top 20 rows



In [ ]:
spark.stop()